In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import joblib
import pickle
from datetime import datetime, timedelta
import random

In [5]:
import re

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

### 1.Банкны зээлийн бүтээгдэхүүнд үндэслэж асуултанд хариулах систем

In [6]:
BANK_PRODUCTS_JSON = {
    "loans": [
        {
            "loan_type": "Эргэлтийн хөрөнгийн зээл",
            "description": "Бизнесийн хэвийн үйл ажиллагааг хангах, бизнесээ өргөжүүлэх зорилгоор эргэлтийн хөрөнгөө санхүүжүүлэх зээл",
            "max_amount": "500,000,000 MNT",
            "term_months": 36,
            "interest_rate": "21.6%-24.0%",
            "purpose": "Бизнесээ өргөжүүлэх, эргэлтийн хөрөнгөө нэмэгдүүлэх",
            "keywords": ["эргэлтийн хөрөнгө", "бизнес", "өргөжүүлэх", "худалдаа", "үйл ажиллагаа"],
            "requirements": ["Бизнесийн орлоготой", "6 сараас дээш туршлага", "барьцаатай"]
        },
        {
            "loan_type": "Үндсэн хөрөнгийн зээл",
            "description": "Тоног төхөөрөмж, барилга байгууламж зэрэг үндсэн хөрөнгө худалдан авахад зориулсан зээл",
            "max_amount": "2,500,000,000 MNT",
            "term_months": 60,
            "interest_rate": "19.2%-24.0%",
            "purpose": "Тоног төхөөрөмж, барилга байгууламж худалдан авах",
            "keywords": ["үндсэн хөрөнгө", "тоног төхөөрөмж", "барилга", "хөрөнгө оруулалт"],
            "requirements": ["Бизнесийн орлоготой", "6 сараас дээш туршлага", "барьцаатай"]
        },
        {
            "loan_type": "Эмэгтэй бизнес эрхлэгчдийг дэмжих зээл",
            "description": "Эмэгтэй бизнес эрхлэгчдэд илүү хөнгөлөлттэй хүүгээр олгодог зээл",
            "max_amount": "500,000,000 MNT",
            "term_months": 60,
            "interest_rate": "19.2%-20.4%",
            "purpose": "Эмэгтэй бизнесменүүдийг дэмжих",
            "keywords": ["эмэгтэй", "бизнес эрхлэгч", "хөнгөлөлттэй", "дэмжлэг"],
            "requirements": ["Эмэгтэй бизнес эрхлэгч", "Орлоготой", "50 сая хүртэл барьцаагүй"]
        },
        {
            "loan_type": "ПОС орлого барьцаалсан зээл",
            "description": "ПОС терминалын орлогоо барьцаалж авах богино хугацааны зээл",
            "max_amount": "50,000,000 MNT",
            "term_months": 12,
            "interest_rate": "22.8%",
            "purpose": "Богино хугацааны эргэлтийн хөрөнгө",
            "keywords": ["пос", "терминал", "богино хугацаа", "орлого барьцаа"],
            "requirements": ["ПОС 6 сараас дээш ашигласан", "Мастер гэрээтэй", "Сайн кредитийн түүх"]
        },
        {
            "loan_type": "Хөдөө аж ахуйн зээл",
            "description": "Хүнс, хөдөө аж ахуйн үйлдвэрлэлийг дэмжих хөнгөлөлттэй зээл",
            "max_amount": "5,000,000,000 MNT",
            "term_months": 36,
            "interest_rate": "5%",
            "purpose": "Хөдөө аж ахуй, тариалангийн үйлдвэрлэл",
            "keywords": ["хөдөө аж ахуй", "тариалан", "мал аж ахуй", "хүнс үйлдвэрлэл"],
            "requirements": ["Хөдөө аж ахуйн чиглэл", "12 сараас дээш үйл ажиллагаа", "Өргүй"]
        },
        {
            "loan_type": "Органик Монгол хөтөлбөр",
            "description": "Газар тариалан, органик ногоо тариаланчдад зориулсан хөнгөлөлттэй зээл",
            "max_amount": "60,000,000 MNT",
            "term_months": 36,
            "interest_rate": "12%",
            "purpose": "Органик тариалан, ногоо үйлдвэрлэл",
            "keywords": ["органик", "тариалан", "ногоо", "газар тариалан"],
            "requirements": ["Хөтөлбөрт хамрагдсан", "Органик гэрчилгээтэй"]
        },
        {
            "loan_type": "Амар зээл",
            "description": "Жижиг бизнесийн богино хугацааны хэрэгцээнд зориулсан хялбар зээл",
            "max_amount": "20,000,000 MNT",
            "term_months": 12,
            "interest_rate": "22%",
            "purpose": "Жижиг бизнесийн хэрэгцээ",
            "keywords": ["амар", "жижиг бизнес", "хялбар", "богино хугацаа"],
            "requirements": ["6 сараас дээш бизнес", "Барьцаагүй"]
        },
        {
            "loan_type": "Өрхийн зээл",
            "description": "Өрхийн хэрэгцээ, боловсрол, эмнэлэг зэрэгт зориулсан хувийн зээл",
            "max_amount": "30,000,000 MNT",
            "term_months": 36,
            "interest_rate": "24%",
            "purpose": "Өрхийн хэрэгцээ, боловсрол, эмнэлэг",
            "keywords": ["өрхийн хэрэгцээ", "боловсрол", "эмнэлэг", "хувийн зээл"],
            "requirements": ["Тогтмол орлоготой", "Иргэн"]
        },
        {
            "loan_type": "Цалингийн зээл",
            "description": "Цалинтай ажилтнуудад зориулсан богино хугацааны зээл",
            "max_amount": "5,000,000 MNT",
            "term_months": 12,
            "interest_rate": "20%",
            "purpose": "Цалинтай ажилтны хэрэгцээ",
            "keywords": ["цалин", "ажилтан", "тогтмол орлого"],
            "requirements": ["Цалингийн данстай", "6 сарын тогтмол орлого"]
        }
    ],
    "qa_pairs": [
        {
            "question": "бизнесийн зээл авах",
            "answer": "Бизнесийн зээлд эргэлтийн хөрөнгийн зээл болон үндсэн хөрөнгийн зээл байна. Эргэлтийн хөрөнгийн зээл нь 500 сая хүртэл, 36 сар хүртэлх хугацаатай, 21.6-24% хүүтэй. Үндсэн хөрөнгийн зээл нь 2.5 тэрбум хүртэл, 60 сар хүртэл, 19.2-24% хүүтэй."
        },
        {
            "question": "эмэгтэй бизнес эрхлэгчийн зээл",
            "answer": "Эмэгтэй бизнес эрхлэгчдэд тусгайлан зориулсан хөнгөлөлттэй зээл байна. 500 сая хүртэл, 60 сар хүртэл, 19.2-20.4% хүүтэй. 50 сая хүртэл барьцаагүй авах боломжтой."
        },
        {
            "question": "хөдөө аж ахуйн зээл",
            "answer": "Хөдөө аж ахуйн зээл нь хамгийн хөнгөлөлттэй 5% жилийн хүүтэй. 5 тэрбум төгрөг хүртэл, 36 сар хүртэлх хугацаатай. Хаврын тариалалт, намрын ургац хураалтанд зориулагдсан."
        },
        {
            "question": "жижиг бизнесийн зээл",
            "answer": "Жижиг бизнест Амар зээл санал болгож байна. 20 сая хүртэл, 12 сар хүртэл, 22% хүү. Барьцаа шаардлагагүй, 6 сараас дээш бизнес эрхэлсэн байх хэрэгтэй."
        },
        {
            "question": "өрхийн хэрэгцээний зээл",
            "answer": "Өрхийн зээл нь 30 сая хүртэл, 36 сар хүртэл, 24% хүүтэй. Боловсрол, эмчилгээ, өрхийн хэрэгцээнд ашиглах боломжтой."
        },
        {
            "question": "цалингийн зээл авах",
            "answer": "Цалинтай ажилчдад цалингийн зээл байна. 5 сая хүртэл, 12 сар, 20% хүү. Банкинд цалингийн данстай, 6 сарын тогтмол орлоготой байх шаардлагатай."
        },
        {
            "question": "пос зээл",
            "answer": "ПОС орлого барьцаалсан зээл нь 50 сая хүртэл, 12 сар, 22.8% хүү. ПОСыг 6 сараас дээш ашигласан, мастер гэрээтэй байх хэрэгтэй."
        },
        {
            "question": "органик тариаланы зээл",
            "answer": "Органик Монгол хөтөлбөрийн зээл 60 сая хүртэл, 36 сар, 12% хүү. Органик тариалан, ногоо үйлдвэрлэлд зориулагдсан."
        }
    ]
}

Нийт 9 төрлийн зээлийн бүтээгдэхүүн дээр тулгуурлаж json үүсгэсэн. Тухайн бүтээгдэхүүний талаар асуухад хариулт өгнө

In [7]:
import json
with open('xac.json', 'w', encoding='utf-8') as f:
    json.dump(BANK_PRODUCTS_JSON, f, ensure_ascii=False, indent=2)

xac.json нэртэй файл үүсч, BANK_PRODUCTS_JSON объект JSON хэлбэрээр хадгалагдав

In [ ]:
class BankingQASystem:
    def __init__(self, products_data=None):
        if products_data is None:
            products_data = BANK_PRODUCTS_JSON
        
        self.products = products_data["loans"]
        self.qa_pairs = products_data["qa_pairs"]
        self.vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
        self.question_vectors = None
        self.answers = []
        self.setup_qa_system()
    
    def preprocess_mongolian_text(self, text):
        """Монгол текст боловсруулах"""
        if not isinstance(text, str):
            return ""
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        return text.strip()
    
    def setup_qa_system(self):
        """Q&A системийг бэлтгэх"""
        questions = []
        answers = []
        
        for qa in self.qa_pairs:
            questions.append(self.preprocess_mongolian_text(qa["question"]))
            answers.append(qa["answer"])
        
        for product in self.products:
            for keyword in product["keywords"]:
                keyword_question = self.preprocess_mongolian_text(keyword + " зээл")
                questions.append(keyword_question)
                
                answer = (f"{product['loan_type']}: {product['description']} "
                         f"Дээд хэмжээ: {product['max_amount']}, "
                         f"Хугацаа: {product['term_months']} сар, "
                         f"Хүү: {product['interest_rate']}. "
                         f"Шаардлага: {', '.join(product['requirements'])}")
                answers.append(answer)
        
        for product in self.products:
            loan_type_question = self.preprocess_mongolian_text(product['loan_type'])
            questions.append(loan_type_question)
            
            answer = (f"{product['loan_type']}: {product['description']} "
                     f"Дээд хэмжээ: {product['max_amount']}, "
                     f"Хугацаа: {product['term_months']} сар, "
                     f"Хүү: {product['interest_rate']}. "
                     f"Зориулалт: {product['purpose']}")
            answers.append(answer)
        
        if questions:
            self.question_vectors = self.vectorizer.fit_transform(questions)
            self.answers = answers
        else:
            raise ValueError("No questions found to train the system")
        
        print(f" Q&A систем бэлтгэгдлээ: {len(questions)} асуулт")
    
    def get_answer(self, question):
        """Асуултанд хариулт өгөх"""
        if not question or not isinstance(question, str):
            return {
                "answer": "Асуулт оруулна уу.",
                "confidence": 0.0,
                "recommended_product": None
            }
        
        processed_question = self.preprocess_mongolian_text(question)
        
        if not processed_question:
            return {
                "answer": "Асуулт тодорхойгүй байна.",
                "confidence": 0.0,
                "recommended_product": None
            }
        
        try:
            question_vector = self.vectorizer.transform([processed_question])
            
            # Calculate similarities
            similarities = cosine_similarity(question_vector, self.question_vectors)
            best_match_idx = np.argmax(similarities)
            confidence = float(similarities[0][best_match_idx])
            
            # Find best matching product
            best_product = self.find_best_product(processed_question)
            
            if confidence > 0.1:  # Minimum confidence threshold
                return {
                    "answer": self.answers[best_match_idx],
                    "confidence": confidence,
                    "recommended_product": best_product
                }
            else:
                return {
                    "answer": "Уучлаарай, таны асуултад тохирсон хариулт олдсонгүй. Банкны ажилтантай холбогдоно уу.",
                    "confidence": confidence,
                    "recommended_product": best_product
                }
                
        except Exception as e:
            return {
                "answer": f"Алдаа гарлаа: {str(e)}",
                "confidence": 0.0,
                "recommended_product": None
            }
    
    def find_best_product(self, question):
        """Асуултанд хамгийн тохирох бүтээгдэхүүнийг олох"""
        best_score = 0
        best_product = None
        
        for product in self.products:
            score = 0
            
            # Check keywords
            for keyword in product["keywords"]:
                if keyword.lower() in question.lower():
                    score += 3
             
            # Check loan type
            if product["loan_type"].lower() in question.lower():
                score += 5
            
            # Check purpose words
            purpose_words = product["purpose"].lower().split()
            for word in purpose_words:
                if len(word) > 2 and word in question.lower():
                    score += 1
            
            # Check description words  
            desc_words = product["description"].lower().split()
            for word in desc_words:
                if len(word) > 3 and word in question.lower():
                    score += 1
            
            if score > best_score:
                best_score = score
                best_product = product
        
        return best_product
    
    def get_all_products(self):
        """Бүх бүтээгдэхүүний жагсаалт"""
        return self.products
    
    def search_products(self, query):
        """Бүтээгдэхүүн хайх"""
        query_lower = query.lower()
        matching_products = []
        
        for product in self.products:
            # Check in loan type
            if query_lower in product["loan_type"].lower():
                matching_products.append(product)
                continue
                
            # Check in keywords
            for keyword in product["keywords"]:
                if query_lower in keyword.lower():
                    matching_products.append(product)
                    break
            
            # Check in description
            if query_lower in product["description"].lower():
                matching_products.append(product)
        
        return matching_products

Банкны 9 төрлийн зээлийн бүтээгдэхүүнд зориулж асуулт хариултын системийг боловсруулсан. 

__init__:
- json-ыг аваад, үүнээсээ тодорхойлж байна: Зээлийн бүтээгдэхүүний жагсаалт, Асуулт хариулт pairs, Харилцагч нарын асуултуудыг тоон хэлбэрт хөрвүүлэх зорилгоор vectorizer, тухайн асуултад харилах текстүүд

preprocess_mongolian_text:
- Үүнд монгол текстүүдийн format хийсэн: Жижиг үсэг болгох, тэмдэгтүүдийг арилгах, оройн болон эхний хоосон зайг алга болгох

set_qa_system:
- self.qa_pairs болон self.products-ын keywords, loan_type зэргийг ашиглан асуултуудыг бэлтгэсэн. 
- Бүтээгдэхүүний мэдээллүүдийг хариултад оруулсан (зээл төрөл, дээд хэмжээ, хугацаа, хүү, шаардлага).
- Бэлтгэсэн асуултуудыг TF-IDF vector болгон хөрвүүлсэн (fit_transform)
- Хэрвээ асуулт байхгүй бол ValueError гаргана.

get_answer:
- Хэрэглэгчийн асуултыг боловсруулна (preprocess_mongolian_text).
- TF-IDF вектор болгон хөрвүүлнэ.
- cosine_similarity ашиглан хамгийн төстэй асуултыг олж хариулт гаргана.
- confidence (итгэлцүүр) тооцно.
- find_best_product функц ашиглан тухайн асуултад хамгийн тохирох бүтээгдэхүүнийг олно.
- Хэрвээ итгэлцүүр бага байвал "хариулт олдсонгүй" гэсэн мессеж харуулна.

find_best_product:
- Хэрэглэгчийн асуулт болон бүтээгдэхүүний keywords, loan_type, purpose, description-г харьцуулна.
- Оноо (score) өгч хамгийн өндөр оноотой бүтээгдэхүүнийг сонгоно.
- Хамгийн тохирох бүтээгдэхүүн return хийнэ.

Дүгнэлт
- Банкны бүтээгдэхүүн болон Q&A өгөгдлийг бэлтгэнэ.
- TF-IDF ашиглан асуултыг тоон хэлбэрт хөрвүүлнэ.
- Асуултанд хамгийн тохирох хариулт болон бүтээгдэхүүнийг буцаана.
- Монгол текст боловсруулах, keyword matching болон cosine similarity ашиглана.

In [9]:
qa_system = BankingQASystem()

 Q&A систем бэлтгэгдлээ: 53 асуулт


In [10]:
print("\nСистемийг туршиж байна...")
test_questions = [
    "бизнесийн зээл авах",
    "эмэгтэй бизнес эрхлэгчийн зээл",
    "хөдөө аж ахуйн зээл",
    "жижиг бизнесийн зээл",
    "цалингийн зээл"
]

for question in test_questions:
    result = qa_system.get_answer(question)
    print(f" {question}")
    print(f" {result['answer'][:100]}...")
    print(f" Итгэлцүүр: {result['confidence']:.3f}")
    print("-" * 50)


Системийг туршиж байна...
 бизнесийн зээл авах
 Бизнесийн зээлд эргэлтийн хөрөнгийн зээл болон үндсэн хөрөнгийн зээл байна. Эргэлтийн хөрөнгийн зээл...
 Итгэлцүүр: 1.000
--------------------------------------------------
 эмэгтэй бизнес эрхлэгчийн зээл
 Эмэгтэй бизнес эрхлэгчдэд тусгайлан зориулсан хөнгөлөлттэй зээл байна. 500 сая хүртэл, 60 сар хүртэл...
 Итгэлцүүр: 1.000
--------------------------------------------------
 хөдөө аж ахуйн зээл
 Хөдөө аж ахуйн зээл нь хамгийн хөнгөлөлттэй 5% жилийн хүүтэй. 5 тэрбум төгрөг хүртэл, 36 сар хүртэлх...
 Итгэлцүүр: 1.000
--------------------------------------------------
 жижиг бизнесийн зээл
 Жижиг бизнест Амар зээл санал болгож байна. 20 сая хүртэл, 12 сар хүртэл, 22% хүү. Барьцаа шаардлага...
 Итгэлцүүр: 1.000
--------------------------------------------------
 цалингийн зээл
 Цалингийн зээл: Цалинтай ажилтнуудад зориулсан богино хугацааны зээл Дээд хэмжээ: 5,000,000 MNT, Хуг...
 Итгэлцүүр: 1.000
------------------------------------------

In [11]:
os.makedirs('models', exist_ok=True)

In [12]:
joblib.dump(qa_system.vectorizer, 'models/qa_vectorizer.pkl')
joblib.dump(qa_system.question_vectors, 'models/question_vectors.pkl')
joblib.dump(qa_system.answers, 'models/qa_answers.pkl')
joblib.dump(qa_system.products, 'models/bank_products.pkl')
joblib.dump(qa_system.qa_pairs, 'models/qa_pairs.pkl')
joblib.dump(qa_system, 'models/banking_qa_system.pkl')

['models/banking_qa_system.pkl']

In [13]:
class BankingQAPredictor:
    def __init__(self):
        self.load_system()
    
    def load_system(self):
        """Load the saved Q&A system"""
        try:
            self.qa_system = joblib.load('models/banking_qa_system.pkl')
            print("Q&A систем амжилттай ачаалагдлаа!")
        except Exception as e:
            print(f"Q&A систем ачаалахад алдаа: {e}")
            # Fallback: create new system
            self.qa_system = BankingQASystem()
    
    def ask_question(self, question):
        """Ask a question and get answer"""
        return self.qa_system.get_answer(question)
    
    def search_loans(self, query):
        """Search for loan products"""
        return self.qa_system.search_products(query)
    
    def get_all_loans(self):
        """Get all available loan products"""
        return self.qa_system.get_all_products()

Энэ class нь хадгалагдсан Q&A системийг дахин ачаалж, харилцагчаас асуулт авч хариулт өгөх, зээлийн бүтээгдэхүүн хайх үйлдлүүдийг хөнгөвчлөх зориулалттай.

In [14]:
predictor = BankingQAPredictor()
joblib.dump(predictor, 'models/banking_qa_predictor.pkl')

print("\nБҮТЭЭГДЭХҮҮНИЙ ЖАГСААЛТ:")
print("=" * 60)
for i, product in enumerate(qa_system.products, 1):
    print(f"{i}. {product['loan_type']}")
    print(f" Дээд хэмжээ: {product['max_amount']}")
    print(f" Хугацаа: {product['term_months']} сар")
    print(f" Хүү: {product['interest_rate']}")
    print(f" Зориулалт: {product['purpose']}")
    print()

Q&A систем амжилттай ачаалагдлаа!

БҮТЭЭГДЭХҮҮНИЙ ЖАГСААЛТ:
1. Эргэлтийн хөрөнгийн зээл
 Дээд хэмжээ: 500,000,000 MNT
 Хугацаа: 36 сар
 Хүү: 21.6%-24.0%
 Зориулалт: Бизнесээ өргөжүүлэх, эргэлтийн хөрөнгөө нэмэгдүүлэх

2. Үндсэн хөрөнгийн зээл
 Дээд хэмжээ: 2,500,000,000 MNT
 Хугацаа: 60 сар
 Хүү: 19.2%-24.0%
 Зориулалт: Тоног төхөөрөмж, барилга байгууламж худалдан авах

3. Эмэгтэй бизнес эрхлэгчдийг дэмжих зээл
 Дээд хэмжээ: 500,000,000 MNT
 Хугацаа: 60 сар
 Хүү: 19.2%-20.4%
 Зориулалт: Эмэгтэй бизнесменүүдийг дэмжих

4. ПОС орлого барьцаалсан зээл
 Дээд хэмжээ: 50,000,000 MNT
 Хугацаа: 12 сар
 Хүү: 22.8%
 Зориулалт: Богино хугацааны эргэлтийн хөрөнгө

5. Хөдөө аж ахуйн зээл
 Дээд хэмжээ: 5,000,000,000 MNT
 Хугацаа: 36 сар
 Хүү: 5%
 Зориулалт: Хөдөө аж ахуй, тариалангийн үйлдвэрлэл

6. Органик Монгол хөтөлбөр
 Дээд хэмжээ: 60,000,000 MNT
 Хугацаа: 36 сар
 Хүү: 12%
 Зориулалт: Органик тариалан, ногоо үйлдвэрлэл

7. Амар зээл
 Дээд хэмжээ: 20,000,000 MNT
 Хугацаа: 12 сар
 Хүү: 22%
 Зориу

### 2.Зээлийн өргөдөл дээр дүгнэлт гаргах

In [15]:
def generate_loan_applications():
    """Зээлийн өргөдлийн synthetic data үүсгэх"""
    
    mongolian_names = [
        'Батбаяр', 'Оюунэрдэнэ', 'Баттулга', 'Сайханбаяр', 'Энхбаяр',
        'Мөнхзул', 'Цагаанхуу', 'Бадамсүрэн', 'Алтанцэцэг', 'Одгэрэл'
    ]
    
    applications = []
    
    for i in range(1000):
        app = {
            'application_id': f'APP_{i+1:05d}',
            'customer_name': random.choice(mongolian_names),
            'age': random.randint(22, 65),
            'gender': random.choice(['Эрэгтэй', 'Эмэгтэй']),
            'education': random.choice(['Бүрэн дунд', 'Дээд боловсрол', 'Магистр']),
            'employment_type': random.choice(['Албан хаагч', 'Хувийн хэвшил', 'Бизнес эрхлэгч', 'Freelancer']),
            'monthly_income': random.randint(500000, 10000000),  
            'work_experience_years': random.randint(1, 25),
            'loan_type': random.choice(['Бизнесийн зээл', 'Өрхийн зээл', 'Цалингийн зээл', 'Хөдөө аж ахуйн зээл']),
            'requested_amount': random.randint(1000000, 100000000),  
            'loan_purpose': random.choice(['Бизнес өргөжүүлэх', 'Өрхийн хэрэгцээ', 'Тоног төхөөрөмж авах', 'Барилга засвар']),
            'existing_loans': random.randint(0, 3),
            'credit_score': random.randint(300, 850),
            'collateral_value': random.randint(0, 200000000),  
            'monthly_expenses': random.randint(200000, 3000000),  
            'bank_relationship_years': random.randint(0, 15),
            'location': random.choice(['Улаанбаатар', 'Дархан', 'Эрдэнэт', 'Аймаг'])
        }
        
        # Зээл батлах эсэхийг шийдвэрлэх logic
        app['approval_score'] = calculate_approval_score(app)
        app['approved'] = 1 if app['approval_score'] > 60 else 0
        app['risk_category'] = get_risk_category(app['approval_score'])
        
        applications.append(app)
    
    return pd.DataFrame(applications)


In [16]:
def calculate_approval_score(app):
    """Зээлийн батлалтын оноог тооцоолох"""
    score = 0
    
    # Орлогын фактор
    income_ratio = app['requested_amount'] / app['monthly_income']
    if income_ratio < 10:
        score += 25
    elif income_ratio < 20:
        score += 15
    elif income_ratio < 30:
        score += 5
    
    # Кредитийн түүх
    if app['credit_score'] > 750:
        score += 25
    elif app['credit_score'] > 650:
        score += 15
    elif app['credit_score'] > 550:
        score += 5
    
    # Ажлын туршлага
    if app['work_experience_years'] > 10:
        score += 15
    elif app['work_experience_years'] > 5:
        score += 10
    elif app['work_experience_years'] > 2:
        score += 5
    
    # Боловсрол
    education_scores = {
        'Дээд боловсрол': 10,
        'Магистр': 15,
        'Тэргүүн боловсрол': 5,
        'Бүрэн дунд': 0
    }
    score += education_scores.get(app['education'], 0)
    
    # Барьцаа
    if app['collateral_value'] > app['requested_amount']:
        score += 20
    elif app['collateral_value'] > app['requested_amount'] * 0.5:
        score += 10
    
    # Одоо байгаа зээлийн тоо
    score -= app['existing_loans'] * 5
    
    # Банктай харилцах хугацаа
    if app['bank_relationship_years'] > 5:
        score += 10
    elif app['bank_relationship_years'] > 2:
        score += 5
    
    return max(0, min(100, score))

Энэ функцаар харилцагч нарын зээлийн өргөдлийг үндэслэн **баталгаажуулах оноог (0–100)** тооцоолсон.
- income_ratio:
Зээлийн дээд хэмжээ орлоготой харьцуулахад бага бол оноо өндөр. Харьцаа их байх тусам оноо багасна.
- credit_score:
Өндөр кредит оноо нь зээлийн эрсдэлийг бууруулж, өндөр оноо авчирна.
- work_experience_years:
Туршлагатай хэрэглэгчид илүү найдвартай гэж тооцож оноо нэмнэ
- education:
Магистр, дээд боловсрол зэрэг өндөр боловсролтой хэрэглэгчдэд оноо нэмсэн.
- collateral_value:
Зээлийн барьцаа нь зээлийн эрсдлийг бууруулдаг тул үнэ өндөр байх тусам оноо нэмнэ.
- existing_loans:
Их зээлтэй бол эрсдэл өндөр гэж үзэн оноог хасна.
- bank_relationship_years:
Удаан хугацаанд харилцаж байгаа хэрэглэгчдэд оноо нэмсэн.
- max(0, min(100, score)):
Эцсийн оноог 0–100 хооронд хязгаарлана.


In [17]:
def get_risk_category(score):
    """Эрсдлийн ангиллыг тодорхойлох"""
    if score >= 80:
        return 'Бага эрсдэлтэй'
    elif score >= 60:
        return 'Дунд эрсдэлтэй'
    elif score >= 40:
        return 'Өндөр эрсдэлтэй'
    else:
        return 'Маш өндөр эрсдэлтэй'

In [18]:
class LoanAnalysisSystem:
    def __init__(self, applications_df):
        self.df = applications_df
        self.setup_models()
    
    def setup_models(self):
        """Машин сургалтын загваруудыг бэлтгэх"""
        features = [
            'age', 'monthly_income', 'work_experience_years', 'requested_amount',
            'existing_loans', 'credit_score', 'collateral_value', 'monthly_expenses',
            'bank_relationship_years'
        ]
        
        X = self.df[features]
        y = self.df['approved']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.model.fit(X_train, y_train)
        
        self.features = features
    
    def predict_approval(self, application_data):
        """Зээлийн батлалтыг урьдчилан таамаглах"""
        features_array = np.array([[application_data[feature] for feature in self.features]])
        probability = self.model.predict_proba(features_array)[0][1]
        prediction = self.model.predict(features_array)[0]
        
        return {
            'prediction': 'Батлагдана' if prediction == 1 else 'Батлагдахгүй',
            'probability': float(probability),
            'confidence': 'Өндөр' if probability > 0.8 or probability < 0.2 else 'Дунд'
        }
    
    def get_feature_importance(self):
        """Feature-уудын ач холбогдлыг олох"""
        importance = self.model.feature_importances_
        feature_imp = dict(zip(self.features, importance))
        return sorted(feature_imp.items(), key=lambda x: x[1], reverse=True)
    
    def generate_insights(self):
        """Дүн шинжилгээний ерөнхий дүгнэлт"""
        insights = {
            'total_applications': len(self.df),
            'approval_rate': float(self.df['approved'].mean()),
            'average_requested_amount': float(self.df['requested_amount'].mean()),
            'risk_distribution': self.df['risk_category'].value_counts().to_dict(),
            'top_factors': self.get_feature_importance()[:5],
            'demographics': {
                'age_distribution': {
                    '22-30': len(self.df[(self.df['age'] >= 22) & (self.df['age'] <= 30)]),
                    '31-45': len(self.df[(self.df['age'] >= 31) & (self.df['age'] <= 45)]),
                    '46-65': len(self.df[(self.df['age'] >= 46) & (self.df['age'] <= 65)])
                },
                'gender_distribution': self.df['gender'].value_counts().to_dict(),
                'education_distribution': self.df['education'].value_counts().to_dict()
            }
        }
        
        return insights
    
    def analyze_customer_segment(self, segment_criteria):
        """Харилцагчийн сегментийн дүн шинжилгээ"""
        if segment_criteria == 'high_income':
            segment = self.df[self.df['monthly_income'] > 2000000]
        elif segment_criteria == 'business':
            segment = self.df[self.df['employment_type'] == 'Бизнес эрхлэгч']
        elif segment_criteria == 'young':
            segment = self.df[self.df['age'] < 35]
        else:
            segment = self.df
            
        return {
            'segment_size': len(segment),
            'approval_rate': float(segment['approved'].mean()),
            'average_amount': float(segment['requested_amount'].mean()),
            'risk_distribution': segment['risk_category'].value_counts().to_dict()
        }

Энэхүү class нь хэрэглэгчийн зээлийн өргөдөл, тэдгээрийн шинж чанар дээр үндэслэн **зээлийн батлалтыг урьдчилан таамаглах**, **features-н ач холбогдлыг тодорхойлох**, **ерөнхий дүн шинжилгээ хийх**, мөн **хэрэглэгчийн сегментийн дүн шинжилгээ хийх** зориулалттай.

init:
- applications_df дээр ажиллана
- setup_models() функцыг дуудаж машин сургалтын загварыг бэлтгэнэ

setup_models:
- Зээлийн баталгааны features болон target (approved) тодорхойлно.
- Train-test split хийж өгөгдлийг сургалт, тестэд хуваана.
- **RandomForestClassifier** загварыг боловсруулж сургана.

predict_approval:
- Өгөгдсөн хэрэглэгчийн зээлийн мэдээллээр батлалтыг урьдчилан таамаглана.
- self.features жагсаалтад заасан шинж чанаруудаар дамжин өгөгдлийг загварт оруулах
- predict_proba ашиглан баталгаажуулах магадлалыг олно.
- prediction: "Батлагдана" эсвэл "Батлагдахгүй"
- probability: Баталгаажүүлэх магадлал (0-1)
- Confidence түвшинг өндөр, дунд гэж ангилна.

get_feature_importance:
- RandomForest-ийн feature importance-ийг гаргаж, хамгийн нөлөөтэй 5 features-г тодорхойлно.

In [19]:
loan_applications = generate_loan_applications()
analysis_system = LoanAnalysisSystem(loan_applications)

- Энэ функц нь зээлийн өргөдлийн dataframe үүсгэсэн.
- Dataframe-д хэрэглэгчийн мэдээлэл, зээлийн хүсэлт, орлого, ажил эрхлэлтийн төрөл, зээлийн батлах статус зэрэг баганууд орно. Энэ нь дараа LoanAnalysisSystem-д боловсруулалт хийх input өгөгдөл болно.

In [20]:
insights = analysis_system.generate_insights()
print(f"Нийт өргөдөл: {insights['total_applications']:,}")
print(f"Батлалтын хувь: {insights['approval_rate']:.1%}")

Нийт өргөдөл: 1,000
Батлалтын хувь: 46.1%


In [21]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X = analysis_system.df[analysis_system.features]
y = analysis_system.df['approved']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y_pred = analysis_system.model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("\nClassification Report:\n", classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

print("\nTop feature importance:")
for feature, importance in analysis_system.get_feature_importance()[:5]:
    print(f"{feature}: {importance:.3f}")

Accuracy: 0.85

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.90      0.86       102
           1       0.89      0.81      0.84        98

    accuracy                           0.85       200
   macro avg       0.86      0.85      0.85       200
weighted avg       0.86      0.85      0.85       200


Confusion Matrix:
 [[92 10]
 [19 79]]

Top feature importance:
credit_score: 0.205
requested_amount: 0.187
monthly_income: 0.140
collateral_value: 0.121
work_experience_years: 0.083


## Зээлийн батлалтад зориулсан RandomForest загварын үр дүн

### 1. Загварын нарийвчлал (Accuracy)
- **Accuracy:** 0.85  
  Загвар нийт 85% зөв таамаглаж байна.

### 2. Classification Report
| Class | Precision | Recall | F1-score | Support |
|-------|----------|--------|----------|---------|
| 0 (Батлагдахгүй) | 0.83 | 0.90 | 0.86 | 102 |
| 1 (Батлагдана)   | 0.89 | 0.81 | 0.84 | 98  |
| **Accuracy**     | -    | -    | 0.85 | 200 |
| **Macro Avg**    | 0.86 | 0.85 | 0.85 | 200 |
| **Weighted Avg** | 0.86 | 0.85 | 0.85 | 200 |

- **Precision:** Загвар тухайн ангиллыг зөв таамагласан хувь.  
- **Recall:** Загвар тухайн ангиллыг хэр сайн олж байгааг харуулна.  
- **F1-score:** Precision ба Recall-ийн тэнцвэртэй дундаж.

### 3. Confusion Matrix
- **92:** Батлагдахгүй гэж зөв таамагласан тохиолдлууд  
- **10:** Батлагдахгүй гэж буруу таамагласан (алдаа)  
- **19:** Батлагдана гэж буруу таамагласан (алдаа)  
- **79:** Батлагдана гэж зөв таамагласан  

### 4. Top Feature Importance
| Feature | Importance |
|---------|------------|
| credit_score | 0.205 |
| requested_amount | 0.187 |
| monthly_income | 0.140 |
| collateral_value | 0.121 |
| work_experience_years | 0.083 |

- 'credit_score' хамгийн их нөлөөтэй бөгөөд 20.5% хувьтай байна.

In [22]:
new_application = {
    'age': 30,
    'monthly_income': 2500000,        
    'work_experience_years': 6,
    'requested_amount': 20000000,     
    'existing_loans': 1,
    'credit_score': 720,
    'collateral_value': 25000000,   
    'monthly_expenses': 800000,
    'bank_relationship_years': 3
}

prediction = analysis_system.predict_approval(new_application)

print("Урьдчилсан таамаг:", prediction['prediction'])
print("Магадлал:", round(prediction['probability']*100, 2), "%")
print("Итгэлцүүр:", prediction['confidence'])


Урьдчилсан таамаг: Батлагдана
Магадлал: 63.0 %
Итгэлцүүр: Дунд


### Дата үүсгэв

In [23]:
np.random.seed(42)
n_customers = 5000

customers_data = {
    'customer_id': range(1, n_customers + 1),
    'age': np.random.randint(18, 70, n_customers),
    'income': np.random.lognormal(13.8, 0.6, n_customers), 
    'employment_years': np.random.gamma(2, 3, n_customers).astype(int),
    'education': np.random.choice(['Дунд', 'Дээд', 'Магистр', 'Доктор'], n_customers, p=[0.3, 0.5, 0.15, 0.05]),
    'marital_status': np.random.choice(['Гэрлэсэн', 'Ганц', 'Салсан'], n_customers, p=[0.6, 0.3, 0.1]),
    'location': np.random.choice(['Улаанбаатар', 'Дархан', 'Эрдэнэт', 'Чойр', 'Мөрөн'], n_customers, p=[0.7, 0.1, 0.1, 0.05, 0.05]),
    'loan_amount': np.random.lognormal(15.5, 0.9, n_customers), 
    'loan_term': np.random.choice([12, 24, 36, 48, 60, 72], n_customers, p=[0.1, 0.2, 0.3, 0.25, 0.1, 0.05]),
    'credit_history_months': np.random.gamma(2, 15, n_customers).astype(int),
    'existing_loans': np.random.poisson(1, n_customers), 
    'collateral_value': np.random.lognormal(16.8, 1.1, n_customers), 
    'bank_relationship_years': np.random.gamma(1.5, 3, n_customers).astype(int),
    'monthly_expense': np.random.lognormal(13.2, 0.5, n_customers), 
    'has_savings': np.random.choice([0, 1], n_customers, p=[0.4, 0.6]),
    'credit_score': np.random.normal(650, 100, n_customers).clip(300, 850).astype(int)
}

customers_df = pd.DataFrame(customers_data)

In [24]:
customers_df['debt_to_income'] = customers_df['loan_amount'] / (customers_df['income'] * customers_df['loan_term'] / 12)
customers_df['expense_to_income'] = customers_df['monthly_expense'] / (customers_df['income'] / 12)
customers_df['collateral_coverage'] = customers_df['collateral_value'] / customers_df['loan_amount']

debt_to_income: 
- Энэ харьцаагаар нь зээлдэгчийн нийт зээлийн өрийг жилийн орлогоор харьцуулж үзсэн.
- Өр өндөр байвал батлах магадлал багасах, эрсдэл ихэснэ.
- Зээлдэгчийн санхүүгийн чадамжийг үнэлэхэд хамгийн гол үзүүлэлт.

expense_to_income:
- Зээлийн бус сарын орлогоор хэрэглээний зардлыг харьцуулж үзсэн харьцаа.
- Өндөр харьцаа байвал хэрэглээ их, хуримтлал бага гэсэн үг бөгөөд зээлийн эрсдэл өндөр.
- Зээлийн батлах шийдвэр гаргахад зээлийн чадамжийг үнэлэх бас нэгэн indicator.

collateral_coverage:
- Зээлдэгчийн барьцааны үнийг зээлийн дүнтэйн харьцуулж байна.
- 1.0-аас дээш бол барьцаа зээлийг бүрэн cover хийж байна (аюул багатай).
- 1.0-оос доош бол барьцааны хангалт дутмаг, эрсдэл өндөр.

In [25]:
approval_score = (
    (customers_df['income'] > 800000) * 0.3 +
    (customers_df['credit_score'] > 600) * 0.25 +
    (customers_df['credit_history_months'] > 24) * 0.2 +
    (customers_df['existing_loans'] < 3) * 0.15 +
    (customers_df['collateral_coverage'] > 1.2) * 0.1 +
    (customers_df['has_savings'] == 1) * 0.05 -
    (customers_df['debt_to_income'] > 0.4) * 0.2 -
    (customers_df['expense_to_income'] > 0.7) * 0.15
)

customers_df['loan_approved'] = (approval_score > 0.5).astype(int)

- Орлого (income): 800,000₮-аас дээш бол 0.3 оноо нэмнэ. Өндөр орлого нь батлах магадлалыг өсгөнө.
- Кредит оноо (credit_score): 600-с дээш бол 0.25 оноо нэмнэ. Энэ нь сайн кредит оноо учраас.
- Кредитийн түүх (credit_history_months): 24 сар-с дээш бол 0.2 оноо (тогтвортой түүх.)
- Одоогийн зээл (existing_loans): 3-с бага бол 0.15 оноо (өрийн ачаалал багатай.)
- Барьцааны хангалт (collateral_coverage): 1.2-с дээш бол 0.1 оноо (барьцаа сайн хангалттай.)
- Хуримтлал (has_savings): 1 бол 0.05 оноо (нэмэлт баталгаа.)
- Өрийн харьцаа (debt_to_income): 0.4-с дээш бол 0.2 оноо хасна (өрийн ачаалал өндөр.)
- Зарлага/Орлого харьцаа (expense_to_income): 0.7-с дээш бол 0.15 оноо хасна → хэрэглээ өндөр.
Энд онооны жингүүд нь хувь хэмжээгээр өгөгдсөн (0-1 хооронд) ба +/- шинж чанаруудаар эцсийн оноо тооцогдож байна.

### Эрсдэлтэй гүйлгээ

In [26]:
n_transactions = 20000
transactions_data = {
    'transaction_id': range(1, n_transactions + 1),
    'customer_id': np.random.randint(1, n_customers + 1, n_transactions),
    'amount': np.random.lognormal(11, 1.8, n_transactions), 
    'transaction_type': np.random.choice(['Шилжүүлэг', 'Төлбөр', 'Данс цэнэглэх', 'ATM', 'Online'], n_transactions),
    'location': np.random.choice(['Улаанбаатар', 'Дархан', 'Эрдэнэт', 'Чойр', 'Мөрөн'], n_transactions),
    'merchant_category': np.random.choice(['Супермаркет', 'Шатахуун', 'Кафе', 'Онлайн', 'ATM', 'Банк'], n_transactions),
    'time_hour': np.random.randint(0, 24, n_transactions),
    'day_of_week': np.random.randint(1, 8, n_transactions),
    'is_international': np.random.choice([0, 1], n_transactions, p=[0.9, 0.1])
}

transactions_df = pd.DataFrame(transactions_data)

In [27]:
fraud_score = (
    (transactions_df['amount'] > 500000) * 0.3 +
    ((transactions_df['time_hour'] < 6) | (transactions_df['time_hour'] > 22)) * 0.2 +
    (transactions_df['is_international'] == 1) * 0.25 +
    (transactions_df['day_of_week'].isin([6, 7])) * 0.1 +
    (transactions_df['merchant_category'] == 'Online') * 0.15
)

transactions_df['is_fraud'] = (fraud_score > np.random.uniform(0.4, 0.8, n_transactions)).astype(int)

print(f"{len(customers_df)} харилцагч, {len(transactions_df)} гүйлгээ бэлтгэгдлээ")

5000 харилцагч, 20000 гүйлгээ бэлтгэгдлээ


- Гүйлгээний дүн (amount): 500,000₮-аас дээш бол 0.3 оноо нэмнэ (өндөр дүнтэй гүйлгээ нь эрсдэлтэй.)
- Цаг (time_hour): Шөнийн 22:00–06:00 цагийн хооронд бол 0.2 оноо нэмнэ (шөнийн гүйлгээ нь эрсдэлтэй.)
- Олон улсын гүйлгээ (is_international): 1 бол 0.25 оноо нэмнэ  (өндөр эрсдэлтэй.)
- Долоо хоногийн өдөр (day_of_week): Бямба/Ням бол 0.1 оноо  (ховор цагийн гүйлгээ.)
- Худалдааны төрөл (merchant_category): Онлайн бол 0.15 оноо  (онлайн гүйлгээ нь илүү эрсдэлтэй.)

### Encoders

In [28]:
education_categories = ['Дунд', 'Дээд', 'Магистр', 'Доктор']
education_encoder = LabelEncoder()
education_encoder.fit(education_categories)

LabelEncoder()

In [29]:
marital_categories = ['Гэрлэсэн', 'Ганц', 'Салсан']
marital_encoder = LabelEncoder()
marital_encoder.fit(marital_categories)

LabelEncoder()

In [30]:
location_categories = ['Улаанбаатар', 'Дархан', 'Эрдэнэт', 'Чойр', 'Мөрөн']
location_encoder = LabelEncoder()
location_encoder.fit(location_categories)

LabelEncoder()

In [31]:
txn_type_categories = ['Шилжүүлэг', 'Төлбөр', 'Данс цэнэглэх', 'ATM', 'Online']
txn_type_encoder = LabelEncoder()
txn_type_encoder.fit(txn_type_categories)

LabelEncoder()

In [32]:
merchant_categories = ['Супермаркет', 'Шатахуун', 'Кафе', 'Онлайн', 'ATM', 'Банк']
merchant_encoder = LabelEncoder()
merchant_encoder.fit(merchant_categories)

LabelEncoder()

In [33]:
joblib.dump(education_encoder, 'models/education_encoder.pkl')
joblib.dump(marital_encoder, 'models/marital_encoder.pkl')
joblib.dump(location_encoder, 'models/location_encoder.pkl')
joblib.dump(txn_type_encoder, 'models/txn_type_encoder.pkl')
joblib.dump(merchant_encoder, 'models/merchant_encoder.pkl')

['models/merchant_encoder.pkl']

### Зээлийн эрсдэлийн үнэлгээ

In [34]:
loan_features = customers_df.copy()
loan_features['loan_to_income_ratio'] = loan_features['loan_amount'] / loan_features['income']
loan_features['collateral_to_loan_ratio'] = loan_features['collateral_value'] / loan_features['loan_amount']
loan_features['age_income_interaction'] = loan_features['age'] * loan_features['income'] / 1000000
loan_features['monthly_payment'] = loan_features['loan_amount'] / loan_features['loan_term']
loan_features['payment_to_income'] = loan_features['monthly_payment'] / (loan_features['income'] / 12)

In [35]:
loan_features['education_encoded'] = education_encoder.transform(loan_features['education'])
loan_features['marital_encoded'] = marital_encoder.transform(loan_features['marital_status'])
loan_features['location_encoded'] = location_encoder.transform(loan_features['location'])

In [36]:
loan_feature_columns = [
    'age', 'income', 'employment_years', 'loan_amount', 'loan_term',
    'credit_history_months', 'existing_loans', 'collateral_value',
    'bank_relationship_years', 'monthly_expense', 'has_savings', 'credit_score',
    'loan_to_income_ratio', 'collateral_to_loan_ratio', 'age_income_interaction',
    'monthly_payment', 'payment_to_income', 'education_encoded', 'marital_encoded', 'location_encoded'
]

In [37]:
X_loan = loan_features[loan_feature_columns]
y_loan = loan_features['loan_approved']

In [38]:
X_train_loan, X_test_loan, y_train_loan, y_test_loan = train_test_split(
    X_loan, y_loan, test_size=0.2, random_state=42, stratify=y_loan
)

stratify=y_loan нь батлагдсан/батлагдаагүй зээлүүдийн хувь тест ба сургалтын багцад ижил хувиар хадгалагдана.

In [39]:
loan_scaler = StandardScaler()
X_train_loan_scaled = loan_scaler.fit_transform(X_train_loan)
X_test_loan_scaled = loan_scaler.transform(X_test_loan)

StandardScaler - Багануудыг mean=0, standard deviation=1 болгодог. Өөрөөр хэлбэл утгуудыг стандартжуулна: 
z = (x - mean) / std. Энэ нь утгуудыг эерэг, сөрөг утгуудтай болгоно.


In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [40]:
os.makedirs('data', exist_ok=True)

In [41]:
loan_model = Sequential([
    Dense(256, activation='relu', input_dim=X_train_loan_scaled.shape[1]),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

**Keras Sequential** загвар нь давхаргуудыг дараалалтай нэмдэг. Тус бүрийн layer нь өмнөх layer-аас гаралтай output-ийг input болгон авна. 

Input Layer + Hidden Layer:
- Dense нь бүх input node бүрийг бүх output node-той холбосон layer. fully connected layer
- 256 нь neurons (hidden layers)
- activation='relu' - non-linear ReLU функц ашиглана (0-аас доош утгыг 0 болгоно, эерэг утгыг хэвээр үлдээдэг)
- input_dim=X_train_loan_scaled.shape[1] - input хэмжээ = feature-ийн тоо (стандарчилсан зээлийн өгөгдлөөс). (эхний layer-д хэдэн feature орж ирж байгааг заав)

Hidden Layers:
- 5 hidden layers нь progressively neuron тоог багасгаж feature-ийн abstraction (хийсвэрлэл)-г сайжруулдаг.
- ReLU activation - gradient vanish-ийг багасгаж, non-linear холбоосыг сурна. Gradient Vanish нь backpropagation үед gradient утгууд дамжин өнгөрөх тусам аажмаар жижигрээд тэг рүү ойртох үзэгдэл юм.

Output Layer:
- 1 neuron - батлалт: 0 (батлагдахгүй) эсвэл 1 (батлагдана).
- sigmoid - 0 ба 1-ийн хооронд probability-г гаргана. (зээл батлагдах магадлал)
- Binary classification - 1 output хэрэгтэй (0 буюу 1)

In [42]:
loan_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

compile() - Model-ийг сургалтанд бэлдэх.

optimizer='adam':
- Optimizer нь network-ийн weights-ийг update хийх алгоритм.
- Adam optimizer нь Adaptive Moment Estimation:
- Gradient-ийн first moment (mean) ба second moment (variance)-ийг ашигладаг
- Learning rate-ийг автоматаар тохируулна
- Олон тооны dataset дээр хурдан, тогтвортой сургалт өгдөг
- SGD (stochastic gradient descent)-тай харьцуулбал хурдан ба илүү тогтвортой.

loss='binary_crossentropy':
- Loss function нь model хэр сайн ажиллаж байгааг хэмждэг.
- binary_crossentropy нь 2 ангиллын classification-д тохиромжтой: Батлагдах / Батлагдахгүй (1 / 0)
- Жинхэнэ ба таамагласан магадлалын ялгааг хэмждэг

metrics=['accuracy']:
- Training болон evaluation үед accuracy-г хэмжиж харах.
- Binary classification-д тохиромжтой.

In [43]:
# Train
history = loan_model.fit(
    X_train_loan_scaled, y_train_loan,
    epochs=100, batch_size=64, validation_split=0.2, verbose=0
)

Моделоо сургаж байна. 
- epochs=100: Бүх training dataset-г 100 удаа network-д feed хийж суралцуулав.
- batch_size=64: Training dataset-г 64 sample-ээр хувааж сургав.
- validation_split=0.2: Training датасетийн 20% нь validation-д хуваагдана.
Validation dataset - overfitting болон model-ийн generalization-ийг шалгах.
- verbose=0: Сургалтын явц дэлгэрэнгүй print хийхгүй.

In [45]:
loan_pred = (loan_model.predict(X_test_loan_scaled) > 0.5).astype(int)
loan_accuracy = np.mean(loan_pred.ravel() == y_test_loan)

print(f"Зээлийн модель Accuracy: {loan_accuracy:.3f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step
Зээлийн модель Accuracy: 0.901


loan_model.predict(X_test_loan_scaled):
- Neural network тестийн датад prediction probability гаргана.
- Output нь 0–1 хоорондын тоо (sigmoid-ийн үр дүн). Жишээ: [0.23, 0.87, 0.12, 0.95]

- Probability-г binary label болгон хөрвүүлэв: 0.5 → 1 (батлагдана) ≤0.5 → 0 (батлагдахгүй)

astype(int):
- True/False → 0/1 болгон хувиргав.

In [46]:
loan_model.save('models/loan_model.h5')
joblib.dump(loan_scaler, 'models/loan_scaler.pkl')
joblib.dump(loan_feature_columns, 'models/loan_feature_columns.pkl')

['models/loan_feature_columns.pkl']

### 5. Customer Segmentation

In [47]:
clustering_features = ['age', 'income', 'employment_years', 'bank_relationship_years', 
                      'existing_loans', 'monthly_expense', 'credit_score']
X_cluster = customers_df[clustering_features]

In [48]:
cluster_scaler = StandardScaler()
X_cluster_scaled = cluster_scaler.fit_transform(X_cluster)

In [49]:
kmeans = KMeans(n_clusters=6, random_state=42)
customers_df['segment'] = kmeans.fit_predict(X_cluster_scaled)

In [50]:
joblib.dump(kmeans, 'models/kmeans_model.pkl')
joblib.dump(cluster_scaler, 'models/cluster_scaler.pkl')
joblib.dump(clustering_features, 'models/clustering_features.pkl')

['models/clustering_features.pkl']

### 6. Fraud Detection

In [51]:
fraud_features = transactions_df.copy()
fraud_features['is_weekend'] = fraud_features['day_of_week'].isin([6, 7]).astype(int)
fraud_features['is_night'] = ((fraud_features['time_hour'] < 6) | (fraud_features['time_hour'] > 22)).astype(int)
fraud_features['amount_log'] = np.log1p(fraud_features['amount'])
fraud_features['amount_zscore'] = (fraud_features['amount'] - fraud_features['amount'].mean()) / fraud_features['amount'].std()

- Амралтын өдрүүдээр feature гаргав.
- Оройн цагийн feature гаргав. 22:00 цагаас хойш болон 06:00-аас өмнө бол 1, бусад нь 0. Учир нь шөнө хийгдсэн гүйлгээнүүд suspicious байдаг.
- 'amount_log' - log transform хийв. np.log1p(x) = log(1+x) = log transform хийхдээ x=0 үед асуудалгүй болгоно. Том дүнтэй гүйлгээнүүдийг шахаж “distribution-г жигд” болгох зорилготой.
- 'amount_zscore' - Standardization / Z-score normalization хийв. Дундажийг 0, стандарт хазайлтыг 1 болгож хувиргана. Ингэснээр их хэмжээгээр зөрсөн гүйлгээнүүд “аномаль” гэж илүү тод харагдана.

In [52]:
fraud_features['txn_type_encoded'] = txn_type_encoder.transform(fraud_features['transaction_type'])
fraud_features['location_encoded'] = location_encoder.transform(fraud_features['location'])
fraud_features['merchant_encoded'] = merchant_encoder.transform(fraud_features['merchant_category'])

In [53]:
fraud_feature_columns = [
    'amount', 'time_hour', 'day_of_week', 'is_weekend', 'is_night', 
    'amount_log', 'amount_zscore', 'is_international',
    'txn_type_encoded', 'location_encoded', 'merchant_encoded'
]

In [54]:
X_fraud = fraud_features[fraud_feature_columns]
y_fraud = fraud_features['is_fraud']

In [60]:
X_fraud

,amount,time_hour,day_of_week,is_weekend,is_night,amount_log,amount_zscore,is_international,txn_type_encoded,location_encoded,merchant_encoded
0,35461.041789,6,7,1,0,10.476218,-0.189872,0,4,3,3
1,996763.205300,15,3,0,0,13.812270,0.493367,0,4,0,3
2,26869.017875,5,6,1,1,10.198766,-0.195979,0,1,1,3
3,331594.562100,1,1,0,1,12.711671,0.020603,0,0,0,1
4,94913.432448,0,5,0,1,11.460731,-0.147617,0,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...
19995,44293.974636,1,1,0,1,10.698627,-0.183594,0,0,2,0
19996,74002.275087,3,6,1,1,11.211865,-0.162479,0,4,1,0
19997,38769.322848,7,2,0,0,10.565410,-0.187521,0,1,0,0
19998,189439.566233,10,7,1,0,12.151831,-0.080433,0,2,4,4


In [55]:
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(
    X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud
)

In [56]:
rf_fraud = RandomForestClassifier(n_estimators=200, random_state=42, max_depth=10)
gb_fraud = GradientBoostingClassifier(n_estimators=200, random_state=42, max_depth=6)

Random Forest Classifier:
- 200 мод үүсгэж, мод тус бүр нь 10 depths (гүн)-тэй.
- олон trees үүсгэж, тэдгээрийн дундаж хариугаас prediction хийнэ.

Gradient Boosting Classifier:
- 200 мод дараалж суралцана. мод тус бүр нь 6 depths-тэй.
- Моднууд дараалалтайгаар суралцана, өмнөхийн алдааг засна.

In [57]:
rf_fraud.fit(X_train_fraud, y_train_fraud)
gb_fraud.fit(X_train_fraud, y_train_fraud)

,loss,'log_loss'
,learning_rate,0.1
,n_estimators,200
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,6
,min_impurity_decrease,0.0
,init,None


In [58]:
rf_pred = rf_fraud.predict_proba(X_test_fraud)[:, 1]
gb_pred = gb_fraud.predict_proba(X_test_fraud)[:, 1]
ensemble_pred = ((rf_pred + gb_pred) / 2 > 0.5).astype(int)
fraud_accuracy = np.mean(ensemble_pred == y_test_fraud)

print(f"Fraud Detection Accuracy: {fraud_accuracy:.3f}")

Fraud Detection Accuracy: 0.980


ensemble_pred:
- Хоёр model-ийн prediction-ийн дундаж магадлал авна.
- Хэрэв дундаж магадлал > 0.5 бол 1 (fraud), үгүй бол 0 (normal).

Хоёр моделийг нийлүүлсэн. Accuracy score: 98% гарсан.

In [ ]:
joblib.dump(rf_fraud, 'models/rf_fraud.pkl')
joblib.dump(gb_fraud, 'models/gb_fraud.pkl')
joblib.dump(fraud_feature_columns, 'models/fraud_feature_columns.pkl')

['models/fraud_feature_columns.pkl']

### Saving Sample Data

In [61]:
sample_customers = customers_df.sample(100, random_state=42)
sample_transactions = transactions_df.sample(200, random_state=42)

In [62]:
sample_customers.to_csv('data/sample_customers.csv', index=False)
sample_transactions.to_csv('data/sample_transactions.csv', index=False)

In [63]:
encoder_info = {
    'education_categories': education_categories,
    'marital_categories': marital_categories,
    'location_categories': location_categories,
    'txn_type_categories': txn_type_categories,
    'merchant_categories': merchant_categories
}

In [64]:
joblib.dump(encoder_info, 'models/encoder_info.pkl')

['models/encoder_info.pkl']

### Creating Prediction functions for Streamlit

In [65]:
class BankingAIPredictor:
    def __init__(self):
        self.load_models()
    
    def load_models(self):
        try:
            self.education_encoder = joblib.load('models/education_encoder.pkl')
            self.marital_encoder = joblib.load('models/marital_encoder.pkl')  
            self.location_encoder = joblib.load('models/location_encoder.pkl')
            self.txn_type_encoder = joblib.load('models/txn_type_encoder.pkl')
            self.merchant_encoder = joblib.load('models/merchant_encoder.pkl')
            
            from tensorflow.keras.models import load_model
            self.loan_model = load_model('models/loan_model.h5')
            self.loan_scaler = joblib.load('models/loan_scaler.pkl')
            self.loan_feature_columns = joblib.load('models/loan_feature_columns.pkl')
            
            self.kmeans = joblib.load('models/kmeans_model.pkl')
            self.cluster_scaler = joblib.load('models/cluster_scaler.pkl')
            self.clustering_features = joblib.load('models/clustering_features.pkl')
            
            self.rf_fraud = joblib.load('models/rf_fraud.pkl')
            self.gb_fraud = joblib.load('models/gb_fraud.pkl')
            self.fraud_feature_columns = joblib.load('models/fraud_feature_columns.pkl')
            
            self.qa_vectorizer = joblib.load('models/qa_vectorizer.pkl')
            self.qa_answers = joblib.load('models/qa_answers.pkl')
            self.question_vectors = joblib.load('models/question_vectors.pkl')
            
            print("Бүх модель амжилттай ачаалагдлаа!")
            
        except Exception as e:
            print(f"Модель ачаалахад алдаа гарлаа: {e}")
    
    def predict_loan_approval(self, customer_data):
        """Зээлийн зөвшөөрөл таамаглах"""
        try:
            features = np.array([
                customer_data['age'],
                customer_data['income'], 
                customer_data['employment_years'],
                customer_data['loan_amount'],
                customer_data['loan_term'],
                customer_data['credit_history_months'],
                customer_data['existing_loans'],
                customer_data['collateral_value'],
                customer_data['bank_relationship_years'],
                customer_data['monthly_expense'],
                customer_data['has_savings'],
                customer_data['credit_score'],
                customer_data['loan_amount'] / customer_data['income'],  
                customer_data['collateral_value'] / customer_data['loan_amount'], 
                customer_data['age'] * customer_data['income'] / 1000000, 
                customer_data['loan_amount'] / customer_data['loan_term'],  
                (customer_data['loan_amount'] / customer_data['loan_term']) / (customer_data['income'] / 12), 
                self.education_encoder.transform([customer_data['education']])[0],
                self.marital_encoder.transform([customer_data['marital_status']])[0],
                self.location_encoder.transform([customer_data['location']])[0]
            ]).reshape(1, -1)
            
            features_scaled = self.loan_scaler.transform(features)
            probability = self.loan_model.predict(features_scaled)[0][0]
            
            return {
                'approval_probability': float(probability),
                'recommendation': 'Зөвшөөрөх' if probability > 0.5 else 'Татгалзах',
                'risk_level': 'Бага' if probability > 0.7 else 'Дунд' if probability > 0.3 else 'Өндөр'
            }
            
        except Exception as e:
            return {'error': f"Алдаа гарлаа: {e}"}
    
    def predict_customer_segment(self, customer_data):
        """Харилцагчийн сегмент таамаглах"""
        try:
            features = np.array([
                customer_data['age'],
                customer_data['income'],
                customer_data['employment_years'], 
                customer_data['bank_relationship_years'],
                customer_data['existing_loans'],
                customer_data['monthly_expense'],
                customer_data['credit_score']
            ]).reshape(1, -1)
            
            features_scaled = self.cluster_scaler.transform(features)
            segment = self.kmeans.predict(features_scaled)[0]
            
            segment_names = {
                0: 'Шинэ харилцагч',
                1: 'Стандарт харилцагч', 
                2: 'Premium харилцагч',
                3: 'VIP харилцагч',
                4: 'Эрсдэлтэй харилцагч',
                5: 'Өндөр орлоготой харилцагч'
            }
            
            return {
                'segment_id': int(segment),
                'segment_name': segment_names.get(segment, f'Сегмент {segment}'),
                'description': f'Таны харилцагчийн сегмент: {segment_names.get(segment, f"Сегмент {segment}")}'
            }
            
        except Exception as e:
            return {'error': f"Алдаа гарлаа: {e}"}
    
    def predict_fraud_risk(self, transaction_data):
        """Fraud эрсдэл таамаглах"""
        try:
            features = np.array([
                transaction_data['amount'],
                transaction_data['time_hour'],
                transaction_data['day_of_week'],
                1 if transaction_data['day_of_week'] in [6, 7] else 0,  
                1 if transaction_data['time_hour'] < 6 or transaction_data['time_hour'] > 22 else 0,  # is_night
                np.log1p(transaction_data['amount']), 
                (transaction_data['amount'] - 50000) / 100000, 
                transaction_data['is_international'],
                self.txn_type_encoder.transform([transaction_data['transaction_type']])[0],
                self.location_encoder.transform([transaction_data['location']])[0],
                self.merchant_encoder.transform([transaction_data['merchant_category']])[0]
            ]).reshape(1, -1)
            
            rf_prob = self.rf_fraud.predict_proba(features)[0][1]
            gb_prob = self.gb_fraud.predict_proba(features)[0][1] 
            avg_prob = (rf_prob + gb_prob) / 2
            
            return {
                'fraud_probability': float(avg_prob),
                'risk_level': 'Өндөр' if avg_prob > 0.7 else 'Дунд' if avg_prob > 0.3 else 'Бага',
                'recommendation': 'Блоклох' if avg_prob > 0.7 else 'Шалгах' if avg_prob > 0.3 else 'Зөвшөөрөх'
            }
            
        except Exception as e:
            return {'error': f"Алдаа гарлаа: {e}"}
    
    def answer_question(self, question):
        """Q&A систем"""
        try:
            question_vector = self.qa_vectorizer.transform([question])
            similarities = cosine_similarity(question_vector, self.question_vectors)[0]
            best_match_idx = np.argmax(similarities)
            confidence = similarities[best_match_idx]
            
            return {
                'answer': self.qa_answers[best_match_idx],
                'confidence': float(confidence),
                'reliable': confidence > 0.3
            }
            
        except Exception as e:
            return {'error': f"Алдаа гарлаа: {e}"}

In [66]:
predictor = BankingAIPredictor()
joblib.dump(predictor, 'models/banking_ai_predictor.pkl')

Бүх модель амжилттай ачаалагдлаа!


['models/banking_ai_predictor.pkl']

In [69]:
print("\nМОДЕЛУУДЫН ҮЗҮҮЛЭЛТ:")
print("=" * 60)
print(f"Loan Risk Model: {loan_accuracy:.1%} accuracy")
print(f"Customer Segments: 6 сегмент үүсгэсэн")
print(f"Fraud Detection: {fraud_accuracy:.1%} accuracy")
print(f"Feature Engineering: {len(loan_feature_columns)} loan features")
print(f"Prediction Functions: Бүх функц бэлэн")


МОДЕЛУУДЫН ҮЗҮҮЛЭЛТ:
Loan Risk Model: 90.1% accuracy
Customer Segments: 6 сегмент үүсгэсэн
Fraud Detection: 98.0% accuracy
Feature Engineering: 20 loan features
Prediction Functions: Бүх функц бэлэн


In [70]:
example_customer = {
    'age': 35,
    'income': 1200000,
    'employment_years': 8,
    'education': 'Дээд',
    'marital_status': 'Гэрлэсэн',
    'location': 'Улаанбаатар',
    'loan_amount': 50000000,
    'loan_term': 60,
    'credit_history_months': 36,
    'existing_loans': 1,
    'collateral_value': 80000000,
    'bank_relationship_years': 5,
    'monthly_expense': 800000,
    'has_savings': 1,
    'credit_score': 720
}

loan_result = predictor.predict_loan_approval(example_customer)
print(f"Зээлийн жишээ: {loan_result['recommendation']} - {loan_result['approval_probability']:.1%}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Зээлийн жишээ: Зөвшөөрөх - 100.0%


In [71]:
segment_result = predictor.predict_customer_segment(example_customer)
print(f"Сегмент жишээ: {segment_result['segment_name']}")

Сегмент жишээ: VIP харилцагч


In [72]:
example_transaction = {
    'amount': 2000000,
    'time_hour': 23,
    'day_of_week': 7,
    'transaction_type': 'Online',
    'location': 'Улаанбаатар',
    'merchant_category': 'Онлайн',
    'is_international': 1
}

fraud_result = predictor.predict_fraud_risk(example_transaction)
print(f"Fraud жишээ: {fraud_result['recommendation']} - {fraud_result['fraud_probability']:.1%}")

Fraud жишээ: Блоклох - 82.1%


In [77]:
qa_result = predictor.answer_question("Зээлийн хүү хэд вэ?")
print(f"Q&A жишээ: {qa_result['answer'][:50]}...")

Q&A жишээ: Бизнесийн зээлд эргэлтийн хөрөнгийн зээл болон үнд...
